In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Final_cleaned.csv')

In [3]:
df.shape

(1070994, 12)

In [4]:
null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), (df == 0).sum()])))
null_zero

,# of null,# of zeros
RECORD,0,0
B,0,0
TAXCLASS,0,0
ZIP,0,0
LTFRONT,0,0
LTDEPTH,0,0
STORIES,0,0
BLDFRONT,0,0
BLDDEPTH,0,0
FULLVAL,0,0


In [5]:
df['ZIP3'] = df.ZIP.apply(lambda x: int(str(x)[:3]))

In [6]:
df.head()

,RECORD,B,TAXCLASS,ZIP,LTFRONT,LTDEPTH,STORIES,BLDFRONT,BLDDEPTH,FULLVAL,AVLAND,AVTOT,ZIP3
0,1,1,4,10004.0,500.0,1046.0,50.0,152.0,207.0,21400000.0,4225500.0,9630000.0,100
1,2,1,4,10004.0,27.0,207.0,50.0,152.0,207.0,193800000.0,14310000.0,87210000.0,100
2,3,1,4,10004.0,709.0,564.0,3.0,709.0,564.0,104686000.0,39008700.0,47108700.0,100
3,4,1,4,10004.0,793.0,551.0,2.0,85.0,551.0,39200000.0,15255000.0,17640000.0,100
4,5,1,4,10004.0,323.0,1260.0,1.0,89.0,57.0,272300000.0,121050000.0,122535000.0,100


In [7]:
null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), (df == 0).sum()])))
null_zero

,# of null,# of zeros
RECORD,0,0
B,0,0
TAXCLASS,0,0
ZIP,0,0
LTFRONT,0,0
LTDEPTH,0,0
STORIES,0,0
BLDFRONT,0,0
BLDDEPTH,0,0
FULLVAL,0,0


In [8]:
def ratios(fullval, avland, avtot, ltf, ltd, blf, bld, sto):
    
    s1 = ltf * ltd
    s2 = blf * bld
    s3 = s2 * sto
    
    return pd.concat([pd.Series(value/ratio) for value in [fullval, avland, avtot] for ratio in [s1,s2,s3]], axis = 1)

In [9]:
import timeit

In [10]:
start = timeit.default_timer()

df[['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']] = ratios(df.FULLVAL.values, df.AVLAND.values,df.AVTOT.values,
                                                                       df.LTFRONT.values,df.LTDEPTH.values,
                                                                      df.BLDFRONT.values,df.BLDDEPTH.values, df.STORIES.values)

print('Total run time: {}s'.format(timeit.default_timer() - start))

Total run time: 0.106990876999987s


In [11]:
df.head()

,RECORD,B,TAXCLASS,ZIP,LTFRONT,LTDEPTH,STORIES,BLDFRONT,BLDDEPTH,FULLVAL,...,ZIP3,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,1,1,4,10004.0,500.0,1046.0,50.0,152.0,207.0,21400000.0,...,100,40.917782,680.142385,13.602848,8.079350,134.296339,2.685927,18.413002,306.064073,6.121281
1,2,1,4,10004.0,27.0,207.0,50.0,152.0,207.0,193800000.0,...,100,34675.254965,6159.420290,123.188406,2560.386473,454.805492,9.096110,15603.864734,2771.739130,55.434783
2,3,1,4,10004.0,709.0,564.0,3.0,709.0,564.0,104686000.0,...,100,261.796157,261.796157,87.265386,97.551991,97.551991,32.517330,117.808271,117.808271,39.269424
3,4,1,4,10004.0,793.0,551.0,2.0,85.0,551.0,39200000.0,...,100,89.714219,836.980890,418.490445,34.913021,325.717946,162.858973,40.371399,376.641401,188.320700
4,5,1,4,10004.0,323.0,1260.0,1.0,89.0,57.0,272300000.0,...,100,669.074647,53676.325646,53676.325646,297.434763,23861.620343,23861.620343,301.083591,24154.346541,24154.346541


### Apply takes too long...

In [17]:
def more_ratios(i,x):
    
    z5 = x[0]
    z3 = x[1]
    tc = x[2]
    boro = x[3]
    ra = x[4]
    
    means = [df[df.ZIP == z5]['r'+str(i)].mean(),
                    df[df.ZIP3 == z3]['r'+str(i)].mean(),
                    df[df.TAXCLASS == tc]['r'+str(i)].mean(),
                    df[df.B == boro]['r'+str(i)].mean(),
                    df['r'+str(i)].mean()]
    
    return [round(ra/mean, 4) for mean in means]

In [18]:
dic = {1: 'r1_', 2:'r2_', 3:'r3_', 4:'r4_', 5:'r5_', 6: 'r6_', 7:'r7_', 8:'r8_', 9:'r9_'}

In [ ]:
start = timeit.default_timer()

for i in range(1,10):
    df[[dic[i]+'1', dic[i]+'2', dic[i]+'3', dic[i]+'4', dic[i]+'5']] \
                = df[['ZIP', 'ZIP3', 'TAXCLASS', 'B', 'r'+str(i)]].apply(lambda x: pd.Series(more_ratios(i, x)), axis = 1)

print('Total run time: {}s'.format(round(timeit.default_timer() - start),3))

### Pandas transform

In [12]:
ratio = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']
group = ['ZIP', 'ZIP3', 'TAXCLASS', 'B', 'ALL']

In [19]:
start = timeit.default_timer()

for r in ratio:
    for g in group:
        if g == 'ALL':
            df[r+'_'+g] = df[r]/df[r].mean()
        else:
            df[r+'_'+g] = df[r]/df.groupby(g)[r].transform('mean')
            
print('Total run time: {}s'.format(round(timeit.default_timer() - start),3))

Total run time: 8s


In [20]:
## transform cannot work on signle columns; must be groupby or whole dataset.
# df.groupby('ZIP')['r1'].transform("mean")

In [21]:
df.head()

,RECORD,B,TAXCLASS,ZIP,LTFRONT,LTDEPTH,STORIES,BLDFRONT,BLDDEPTH,FULLVAL,...,r8_ZIP,r8_ZIP3,r8_TAXCLASS,r8_B,r8_ALL,r9_ZIP,r9_ZIP3,r9_TAXCLASS,r9_B,r9_ALL
0,1,1,4,10004.0,500.0,1046.0,50.0,152.0,207.0,21400000.0,...,1.792742,2.087445,0.900471,2.164448,4.464443,0.107215,0.167600,0.036248,0.175899,0.202067
1,2,1,4,10004.0,27.0,207.0,50.0,152.0,207.0,193800000.0,...,16.235204,18.904061,8.154728,19.601404,40.430327,0.970946,1.517796,0.328264,1.592957,1.829938
2,3,1,4,10004.0,709.0,564.0,3.0,709.0,564.0,104686000.0,...,0.690051,0.803486,0.346603,0.833126,1.718425,0.687808,1.075191,0.232539,1.128434,1.296309
3,4,1,4,10004.0,793.0,551.0,2.0,85.0,551.0,39200000.0,...,2.206142,2.568803,1.108116,2.663562,5.493928,3.298458,5.156192,1.115164,5.411525,6.216588
4,5,1,4,10004.0,323.0,1260.0,1.0,89.0,57.0,272300000.0,...,141.481835,164.739619,71.064456,170.816614,352.330464,423.066028,661.342293,143.032918,694.091742,797.350533


In [22]:
df.columns

Index(['RECORD', 'B', 'TAXCLASS', 'ZIP', 'LTFRONT', 'LTDEPTH', 'STORIES',
       'BLDFRONT', 'BLDDEPTH', 'FULLVAL', 'AVLAND', 'AVTOT', 'ZIP3', 'r1',
       'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r1_ZIP', 'r1_ZIP3',
       'r1_TAXCLASS', 'r1_B', 'r1_ALL', 'r2_ZIP', 'r2_ZIP3', 'r2_TAXCLASS',
       'r2_B', 'r2_ALL', 'r3_ZIP', 'r3_ZIP3', 'r3_TAXCLASS', 'r3_B', 'r3_ALL',
       'r4_ZIP', 'r4_ZIP3', 'r4_TAXCLASS', 'r4_B', 'r4_ALL', 'r5_ZIP',
       'r5_ZIP3', 'r5_TAXCLASS', 'r5_B', 'r5_ALL', 'r6_ZIP', 'r6_ZIP3',
       'r6_TAXCLASS', 'r6_B', 'r6_ALL', 'r7_ZIP', 'r7_ZIP3', 'r7_TAXCLASS',
       'r7_B', 'r7_ALL', 'r8_ZIP', 'r8_ZIP3', 'r8_TAXCLASS', 'r8_B', 'r8_ALL',
       'r9_ZIP', 'r9_ZIP3', 'r9_TAXCLASS', 'r9_B', 'r9_ALL'],
      dtype='object')

In [30]:
final = df.iloc[:, 22:]

In [35]:
null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [final.isna().sum(), (final == 0).sum()])))
null_zero

,# of null,# of zeros
r1_ZIP,0,0
r1_ZIP3,0,0
r1_TAXCLASS,0,0
r1_B,0,0
r1_ALL,0,0
r2_ZIP,0,0
r2_ZIP3,0,0
r2_TAXCLASS,0,0
r2_B,0,0
r2_ALL,0,0


In [32]:
final.shape

(1070994, 45)

In [33]:
final.to_csv('final_data.csv', index = False)